In [1]:
# Import libraries
import RPi.GPIO as GPIO
import adcUtil as adc
import accUtil as acc
import time
import numpy as np
import pigpio
import random
from queue import Queue
import threading
import smtplib, string, subprocess

# Create reference to pigpio daemon
pi = pigpio.pi(port = 8887)
# Set GPIO numbering
GPIO.setmode(GPIO.BCM)

# Define components
class Button():
    ''' Stores button attributes '''
    def __init__(self, color, pin):
        self.color = color
        self.pin = pin

class Led():
    ''' Stores LED attributes '''
    def __init__(self, pin):
        self.pin = pin
    
    def flash():
        GPIO.output(self.pin, True)   # turn on
        time.sleep(0.2)               # wait
        GPIO.output(self.pin, False)  # turn off
        time.sleep(0.5)               # wait

class Buzzer():
    ''' Stores buzzer attributes '''
    def __init__(self, frequency, duty, duration, pin):
        self.frequency = frequency
        self.duty = duty
        self.duration = duration
        self.pin = pin

    def sound():
        pi.hardware_PWM(self.pin, self.frequency, int(self.duty * 1e6)) # turn on
        time.sleep(self.duration)               # wait
        pi.hardware_PWM(self.pin, 0, 0)         # turn off
        time.sleep(self.duration)               # wait

# Initialize LEDS
redLED = Led(27)
blueLED = Led(26)
# Initialize Buttons
greenButton = Button("GREEN", 20)
yellowButton = Button("YELLOW", 21)
blueButton = Button("BLUE", 22)
redButton = Button("RED", 23)
# Initialize Buzzers/mic
microphone = 25
buzzer1 = Buzzer(500, 0.50, 2, 12)
buzzer2 = Buzzer(300, 0.70, 3, 19)

# Set GPIO pin to output mode
# LEDs
GPIO.setup(redLED.pin, GPIO.OUT)
GPIO.setup(blueLED.pin, GPIO.OUT)
# Buttons
GPIO.setup(greenButton.pin, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)
GPIO.setup(blueButton.pin, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)
GPIO.setup(yellowButton.pin, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)
GPIO.setup(redButton.pin, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)
# Microphone
GPIO.setup(microphone, GPIO.IN)

# Generate password
password = np.array([greenButton, blueButton, yellowButton, redButton])
np.random.shuffle(password)
print(password)

ModuleNotFoundError: No module named 'RPi'

In [ ]:
# Hold alert data
class Alert():
    def __init__(self):
        self.startTime = time.time()
        self.duration = 0

    def setDuration():
        self.duration = time.time() - self.startTime

    def setSoundData(self, soundData):
        self.soundData = soundData

    def setMovementData(self, movementData):
        self.movementData = movementData

In [2]:
# Checks for correct password - producer
def checkPassword(Qb, Ql, password):
    passwordQueue = Queue(maxsize = 4)
    buttonQueue = Queue(maxsize = 4)
    # Populate passwordQueue with password
    for i in password:
        passwordQueue.put(i)
    # Add buttons to queue as they are pressed
    while True:
        ### TODO: Constantly pick up button data somehow ###
        if(GPIO.input(redButton.pin) == 1):
            buttonQueue.put(redButton)
            print(redButton.color)
        elif(GPIO.input(blueButton.pin) == 1):
            buttonQueue.put(blueButton)
            print(blueButton.color)
        elif(GPIO.input(yellowButton.pin) == 1):
            buttonQueue.put(yellowButton)
            print(yellowButton.color)
        elif(GPIO.input(greenButton.pin) == 1):
            buttonQueue.put(greenButton)
            print(greenButton.color)
        # Check if buttons match password
        if(buttonQueue.full()):
            if(buttonQueue == passwordQueue):
                # Empty queues
                with Qb.mutex:
                    Qb.queue.clear()
                with Ql.mutex:
                    Ql.queue.clear()
                return True
        # Populate queue
        Qb.put(buzzer1) # start buzzer1
        Qb.put(buzzer2) # start buzzer2
        Ql.put(redLED)  # start redLED
        Ql.put(blueLED) # start blueLED

In [ ]:
# Set off lights - consumer
def flashLight(Q, test):
    while True:
        LED = Q.get()
        LED.flash()
        Q.task_done()

# Set off buzzer alarm - consumer
def soundBuzzer(Q, test):
    while True:
        buzzer = Q.get()
        buzzer.sound()
        Q.task_done()

In [ ]:
# Convert password to color names
def getReadablePassword(password):
    return [password[0].color, password[1].color, password[2].color, password[3].color]

In [ ]:
# Send Email
def sendEmail(email, imap):
    # Form the body of the email
    body = "From: %s" % email + "\nTo: %s" % email + "\nSubject: Alarm Password" + "\n" + "Your Alarm Password is " + getReadablePassword(password)
      
    # Send the email  
    server = smtplib.SMTP('smtp.gmail.com:587')  
    server.starttls()  
    server.login(email, imap)  
    server.sendmail(email, email, body)  
    server.quit()

In [ ]:
alerts = np.array([])
soundData = np.arrange(1)
movementData = np.arrange(1)

In [3]:
def collectActivity(microphoneData, accelerometerData):
    # Read acelerometer data and microphone data and store

# Test for sound and movement
def detectActivity():
    ax,ay,az = acc.readACC()
    acceleration = (ax**2 + ay**2 + az**2)**(1/2)
    # Return when motion/sound detected
    while acceleration < 10 and GPIO.input(microphone) == 0:
        time.sleep(1)
        ax,ay,az = acc.readACC()
        acceleration = (ax**2 + ay**2 + az**2)**(1/2)

# Start all alarm systems
def startAlarm():
    # Send alert email
    #sendEmail("lillianbrewster@mymail.mines.edu", "3mzk&~wy^4EvplO")
    Qb = Queue(maxsize = 20)
    Ql = Queue(maxsize = 20)
    iDontKnowWhyThisHasToBeHereButItDoes = 0
    # Initialize alarm threads
    tP = threading.Thread(target=checkPassword, args=(Qb, Ql, password))    # Producer ~(+4/s)+(+4/s)
    tL = threading.Thread(target=flashLights, args=(Ql, iDontKnowWhyThisHasToBeHereButItDoes))   # Consumer ~(0/s)+(-1/7s)
    tB = threading.Thread(target=soundBuzzer, args=(Qb, iDontKnowWhyThisHasToBeHereButItDoes))   # Consumer ~(-1/10s)+(0/s)
    # Start alarm threads
    tP.start()
    time.sleep(1)
    tL.start()
    tB.start()
    # Keep threads working until queues are empty
    Qb.join()
    Ql.join()

# Consistently run program to collect data
def run():
    while True:
        detectActivity()
        alert = Alert()
        collectActivity()
        startAlarm() # TODO run in parallel with collectActivity()
        alert.setDuration()
        alert.setSoundData(soundData)
        alert.setMovementData(movementData)
        alerts.append(alert)


In [1]:
run()

Exception: File `'().py'` not found.